# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline

In [ ]:
import json
import math
import os
from pathlib import Path
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet
from parcels import AdvectionRK4
from datetime import timedelta, datetime

import utils
from utils import get_file_info, load_config

# ignore annoying deprecation warnings
import warnings
warnings.simplefilter("ignore", UserWarning)
import cartopy

# ignore divide by nan error that happens constantly with parcels
np.seterr(divide='ignore', invalid='ignore')

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    "parcels_configs/tijuana_lin_cfg.json",
    "parcels_configs/tijuana_simpterp_cfg.json",
    "parcels_configs/tijuana_lin_aggr_cfg.json"
]

loaded_configs = [load_config(path) for path in configs]
files = [get_file_info(cfg["name"], cfg["netcdf_path"], cfg["resolution"], cfg["parcels_config"]) for cfg in loaded_configs]

## Animated gif stuff and particle simulation

runs on each file you give it

In [ ]:
# animation man very cool
# reference tutorial_Agulhasparticles
# needs ErrorCode for particle recovery
from operator import attrgetter
from parcels import ErrorCode, JITParticle, Variable

max_v = 1.1 # for display purposes only, so the vector field colors don't change every iteration

class TimedParticle(JITParticle):
    lifetime = Variable("lifetime", initial=0, dtype=np.float32)
    spawntime = Variable("spawntime", initial=attrgetter("time"), dtype=np.float32)
    
def ParticleLifetime(particle, fieldset, time):
    particle.lifetime += particle.dt

def DeleteParticle(particle, fieldset, time):
    particle.delete()
    
def exec_save_pset(data, i, runtime, dt, zpad=3, save_snapshot=True, exec_pset=True):
    """
    Saves a snapshot of a particle simulation and then executes.
    
    Args:
        data (dict)
        i (int)
        runtime (float): seconds
        dt (float): seconds
    """
    if save_snapshot:
        data["pset"].show(savefile=str(data["snap_path"])+"/particles"+str(i).zfill(zpad), field="vector", vmax=max_v)
    
    if exec_pset:
        # temporary - TODO make it only init once
        k_plifetime = data["pset"].Kernel(ParticleLifetime)

        data["pset"].execute(
            AdvectionRK4 + k_plifetime,
            runtime=timedelta(seconds=runtime),
            dt=timedelta(seconds=dt),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
            output_file=data["pfile"]
        )

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
part_path = Path("particledata")
part_path.mkdir(parents=True, exist_ok=True)

for f in files:
    cfg = f["cfg"]
    repeat_dt = timedelta(seconds=cfg["repeat_dt"]) # interval at which particles are released
    cfg["spawn_points"] = np.array(cfg["spawn_points"]) # particle spawns will be randomly chosen between these points
    
    repetitions = math.floor(f["timerng_secs"][1] / repeat_dt.total_seconds())
    # the total number of particles that will exist in the simulation
    total = repetitions * cfg["particles_per_dt"]
    lat_arr = np.zeros(total)
    lon_arr = np.zeros(total)
    time_arr = np.zeros(total)
    for i in range(repetitions):
        time_arr[cfg["particles_per_dt"] * i:cfg["particles_per_dt"] * (i + 1)] = repeat_dt.total_seconds() * i

    # randomly select spawn points from the given config
    sp_lat = cfg["spawn_points"].T[0][np.random.randint(0, len(cfg["spawn_points"]), total)]
    sp_lon = cfg["spawn_points"].T[1][np.random.randint(0, len(cfg["spawn_points"]), total)]
    # vary spawn locations using max_variation
    variances_lat = (np.random.random(total) * 2 - 1) * cfg["max_variation"]
    variances_lon = (np.random.random(total) * 2 - 1) * cfg["max_variation"]

    p_lats = sp_lat + variances_lat
    p_lons = sp_lon + variances_lon

    # set up ParticleSet and ParticleFile
    f["pset"] = ParticleSet(fieldset=f["fs"], pclass=TimedParticle, lon=p_lons, lat=p_lats, time=time_arr)
    save_path = f"{part_path}/particle_{f['name']}.nc"
    f["pfile"] = f["pset"].ParticleFile(save_path)
    print(f"Particle trajectories for {f['name']} will be saved to {save_path}")

In [ ]:
[f["pset"].show(field="vector", vmax=max_v) for f in files]

### simulation setup and execution

simulation parameter setup

In [ ]:
# setting up times, intervals, and paths for simulation
for f in files:
    cfg = f["cfg"]
    cfg["snap_num"] = math.floor((f["timerng_secs"][1] - f["timerng_secs"][0]) / cfg["snapshot_interval"])
    cfg["last_int"] = f["timerng_secs"][1] - cfg["snap_num"] * cfg["snapshot_interval"]
    if cfg["last_int"] == 0:
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 1}")
    else:
        print(f"Num snapshots to save for {f['path']}: {cfg['snap_num'] + 2}")
    f["snap_path"] = Path(f"snapshots_{utils.filename_dict[f['res']]}/{f['name']}")
    f["snap_path"].mkdir(parents=True, exist_ok=True)
    print(f"Path to save snapshots to: {f['snap_path']}")
    # only clear directory if desired or actually saving images
    if cfg["save_snapshots"]:
        for p in f["snap_path"].glob("*.png"):
            p.unlink()

execution of all simulation configurations

In [ ]:
# execution of simulation
for f in files:
    cfg = f["cfg"]
    for i in range(cfg["snap_num"]):
        exec_save_pset(f, i, cfg["snapshot_interval"], cfg["simulation_dt"], save_snapshot=cfg["save_snapshots"])

    # save the second-to-last frame
    exec_save_pset(f, cfg["snap_num"], 0, 0, save_snapshot=cfg["save_snapshots"], exec_pset=False)

    # run the last interval (the remainder) if needed
    if cfg["last_int"] != 0:
        exec_save_pset(f, cfg["snap_num"] + 1, cfg["last_int"], cfg["simulation_dt"], save_snapshot=cfg["save_snapshots"])
        
    f["pfile"].export()
    f["pfile"].close()

print("all simulations done and snapshots saved (if simulation was saving snapshots)")

### gif generation

don't have to run, requires [magick](https://imagemagick.org/index.php)

the gifs will be saved `snapshots_west_coast_xkm_hourly/` where xkm is the resolution

In [ ]:
import subprocess

gif_delay = 25 # ms


for f in files:
    if f["cfg"]["save_snapshots"]:
        magick_sp = subprocess.Popen(
            [
                "magick", "-delay", str(gif_delay),
                str(f["snap_path"]) + "/*.png", # path to the snapshots to stitch
                f"snapshots_{utils.filename_dict[f['res']]}/partsim_{f['name']}.gif" # path to save gif to
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        stdout, stderr = magick_sp.communicate()
        print((stdout, stderr))